VQE Implementation in Qiskit 

This section illustrates an implementation of VQE using the programmatic approach. 
Qiskit Aqua also enables a declarative implementation, however, it reveals less 
information about the underlying algorithm. This code, specifically the preparation 
of qubit operators, is based on the code found in the Qiskit Tutorials repository 
(and as of July 2019, may be found at: https://github.com/Qiskit/qiskit-tutorials ).

The following libraries must first be imported.

In [2]:
!pip list

Package              Version
-------------------- ---------
argon2-cffi          21.3.0
argon2-cffi-bindings 21.2.0
asttokens            2.0.5
attrs                21.4.0
backcall             0.2.0
bleach               4.1.0
certifi              2021.5.30
cffi                 1.15.0
charset-normalizer   2.0.12
cryptography         36.0.2
cycler               0.11.0
debugpy              1.5.1
decorator            5.1.1
defusedxml           0.7.1
dill                 0.3.4
entrypoints          0.4
executing            0.8.3
fonttools            4.31.2
h5py                 3.6.0
idna                 3.3
ipykernel            6.9.1
ipython              8.1.1
ipython-genutils     0.2.0
ipywidgets           7.6.5
jedi                 0.18.1
Jinja2               3.0.3
joblib               1.1.0
jsonschema           4.4.0
jupyter              1.0.0
jupyter-client       7.1.2
jupyter-console      6.4.0
jupyter-core         4.9.2
jupyterlab-pygments  0.1.2
jupyterlab-widgets   1.0.2
kiwisolver   

In [2]:
from qiskit_nature.drivers import PySCFDriver, UnitsType

ModuleNotFoundError: No module named 'qiskit_nature'

In [1]:
from qiskit.algorithms import VQE, NumPyEigensolver
import matplotlib.pyplot as plt
import numpy as np
from qiskit_nature.circuit.library import UCCSD
from qiskit_nature.circuit.library import HartreeFock
from qiskit.circuit.library import EfficientSU2
from qiskit.algorithms.optimizers import COBYLA, SPSA, SLSQP
from qiskit.opflow.primitive_ops import Z2Symmetries
from qiskit import IBMQ, BasicAer, Aer
from qiskit_nature.drivers import PySCFDriver, UnitsType
from qiskit_nature.drivers.second_quantization import PySCFDriver
from qiskit_nature.operators.second_quantization import FermionicOp
from qiskit.utils import QuantumInstance
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter
from qiskit.providers.aer.noise import NoiseModel

ModuleNotFoundError: No module named 'qiskit'

Running VQE on a Statevector Simulator 

We demonstrate the calculation of the ground state energy for LiH at various interatomic distances. A driver for the molecule must be created at each such distance. Note that in this experiment, to reduce the number of qubits used, we freeze the core and remove two unoccupied orbitals. First, we define a function that takes an interatomic distance and returns the appropriate qubit operator,  
H, as well as some other information about the operator.

In [ ]:
def get_qubit_op(dist):
    driver = PySCFDriver(atom="Li .0 .0 .0; H .0 .0 " + str(dist), unit=UnitsType.ANGSTROM, 
                         charge=0, spin=0, basis='sto3g')
    molecule = driver.run()
    freeze_list = [0]
    remove_list = [-3, -2]
    repulsion_energy = molecule.nuclear_repulsion_energy
    num_particles = molecule.num_alpha + molecule.num_beta
    num_spin_orbitals = molecule.num_orbitals * 2
    remove_list = [x % molecule.num_orbitals for x in remove_list]
    freeze_list = [x % molecule.num_orbitals for x in freeze_list]
    remove_list = [x - len(freeze_list) for x in remove_list]
    remove_list += [x + molecule.num_orbitals - len(freeze_list)  for x in remove_list]
    freeze_list += [x + molecule.num_orbitals for x in freeze_list]
    ferOp = FermionicOperator(h1=molecule.one_body_integrals, h2=molecule.two_body_integrals)
    ferOp, energy_shift = ferOp.fermion_mode_freezing(freeze_list)
    num_spin_orbitals -= len(freeze_list)
    num_particles -= len(freeze_list)
    ferOp = ferOp.fermion_mode_elimination(remove_list)
    num_spin_orbitals -= len(remove_list)
    qubitOp = ferOp.mapping(map_type='parity', threshold=0.00000001)
    qubitOp = Z2Symmetries.two_qubit_reduction(qubitOp, num_particles)
    shift = energy_shift + repulsion_energy
    return qubitOp, num_particles, num_spin_orbitals, shift

In [ ]:
backend = BasicAer.get_backend("statevector_simulator")
distances = np.arange(0.5, 4.0, 0.1)
exact_energies = []
vqe_energies = []
optimizer = SLSQP(maxiter=5)

In [ ]:
for dist in distances:
    qubitOp, num_particles, num_spin_orbitals, shift = get_qubit_op(dist)
    result = NumPyEigensolver(qubitOp).run()
    exact_energies.append(np.real(result.eigenvalues) + shift)
    initial_state = HartreeFock(
        num_spin_orbitals,
        num_particles,
        qubit_mapping='parity'
    ) 
    var_form = UCCSD(
        num_orbitals=num_spin_orbitals,
        num_particles=num_particles,
        initial_state=initial_state,
        qubit_mapping='parity'
    )
    vqe = VQE(qubitOp, var_form, optimizer)
    vqe_result = np.real(vqe.run(backend)['eigenvalue'] + shift)
    vqe_energies.append(vqe_result)
    print("Interatomic Distance:", np.round(dist, 2), "VQE Result:", vqe_result, "Exact Energy:", exact_energies[-1])
    
print("All energies have been calculated")